<a href="https://colab.research.google.com/github/lunadacosta/Master-Thesis/blob/master/String_Dataset_Pre_processing_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# String Dataset Pre-processing Methods

Created by: Luciana G. da Costa

A study on pre-processing and vectorization methods of string inputs for the traing of NNs.

Mount Google drive and Load dataset with Pandas. Also spliting the dataset. This part was a test for future use, does ot have big relevancy to this study.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import pandas as pd
##### Load data with Pandas ######

filepath_dict = {'spam': 'gdrive/My Drive/Colab Notebooks/dataset/spam.csv',
                 'ham': 'gdrive/My Drive/Colab Notebooks/dataset/ham.csv'}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['symbols', 'label'], sep=';')
    df['source'] = source  #Add another column filled with the source name
    df_list.append(df)

df = pd.concat(df_list)
print(df.iloc[0:10,:])
### randomize dataset
#frame = df.sample(frac=1).reset_index(drop=True)
#print(frame.iloc[0:10,:])

############## split the dataset in train and test dataset ##############
from sklearn.model_selection import train_test_split

for source in df['source'].unique():
    df_source = df[df['source'] == source]
    sentences = df_source['symbols'].values
    y = df_source['label'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=42)


                                             symbols  label source
0               ARC_NA              DATE_IN_PAST ...      1   spam
1               ARC_NA              DATE_IN_PAST ...      1   spam
2               ARC_NA              FROM_HAS_DN  ...      1   spam
3               ARC_NA              FROM_NEQ_DISP...      1   spam
4               ARC_NA              R_SPF_DNSFAIL...      1   spam
5               ARC_NA              SUBJ_EXCESS_B...      1   spam
6               ARC_NA              R_SPF_DNSFAIL...      1   spam
7               ARC_NA              SUBJECT_ENDS_...      1   spam
8               ARC_NA              FROM_HAS_DN  ...      1   spam
9               ARC_NA              DATE_IN_PAST ...      1   spam


# Word Counts with CountVectorizer

The vectorizer object provided by Scikit-Learn [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) is a very simple way to both tokenize a collection of text documents and build a vocabulary of know words, but also encode new documents using that vocabulary.

Itt returns a sparse vector with the length of the entire vocabulary and a n interger count for the number of times each word appeared in the document.

In [8]:
##### Tokenize using CountVectorizer#######
from sklearn.feature_extraction.text import CountVectorizer
# create the transform  
vectorizer = CountVectorizer()
# tokenize an create the vocabulary
vectorizer.fit(sentences_train)
#summarize
vocab = vectorizer.vocabulary_
print("Vocabulary: ",vocab)
# encode document
X_train = vectorizer.transform(sentences_train)
X_test = vectorizer.transform(sentences_test)
# summarize encoded vector
print("Train vocab shape: ", X_train.shape)
print("Train type: ",type(X_train))
print("Train vector array: ",X_train.toarray())


Vocabulary:  {'arc_na': 0, 'mv_case': 14, 'mime_good': 11, 'to_dn_none': 30, 'dmarc_dnsfail': 2, 'from_no_dn': 7, 'rcpt_count_twelve': 21, 'rcvd_count_zero': 23, 'mime_trace': 12, 'r_dkim_na': 15, 'date_in_past': 1, 'mid_rhs_not_fqdn': 10, 'hfilter_hostname_unknown': 8, 'rcpt_count_one': 18, 'to_dom_eq_from_dom': 32, 'rcpt_count_three': 20, 'fake_reply': 4, 'subj_all_caps': 24, 'rcpt_count_seven': 19, 'rcpt_count_five': 16, 'rcpt_count_two': 22, 'rcpt_count_gt_50': 17, 'to_dn_some': 31, 'suspicious_recips': 29, 'subject_ends_exclaim': 26, 'missing_to': 13, 'introduction': 9, 'empty_subject': 3, 'subject_ends_question': 27, 'freemail_from': 6, 'freemail_envfrom': 5, 'subj_excess_qp': 25, 'subject_has_question': 28, 'to_eq_from': 33}
Train vocab shape:  (752, 34)
Train type:  <class 'scipy.sparse.csr.csr_matrix'>
Train vector array:  [[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 1 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 1 0]]


# Hashing with HashingVectorizer

Counting the frequency can be very helpful, but the problem is that the vocabulary can become very large.

A clever work-around is to use one way hash of strings to convert them to intergers. The clever part of it is that no vocabulary is needed and one can choose an arbitrary-long fixed length vector. The downside of this method is that there is no way to do inverted encoding (decoding).

The [HashingVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) class can be used to implement this approach.

In [9]:
#### Tokenize using Hashing_trick #######
from sklearn.feature_extraction.text import HashingVectorizer

#estimate the size of the vocabulary
vocab_size = len(vectorizer.vocabulary_)
print("Vocab size: ",vocab_size)
#integer encode the document
vectorizer = HashingVectorizer(n_features=vocab_size*2)
#encode document
X_train = vectorizer.transform(sentences_train)
X_test = vectorizer.transform(sentences_test)

print("Train Vocab Shape: ",X_train.shape)
print("Train Vector array: ",X_train.toarray())

Vocab size:  34
Train Vocab Shape:  (752, 68)
Train Vector array:  [[0.33333333 0.         0.         ... 0.         0.         0.        ]
 [0.33333333 0.         0.         ... 0.         0.         0.        ]
 [0.33333333 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.31622777 0.         0.         ... 0.         0.         0.        ]
 [0.31622777 0.         0.         ... 0.         0.         0.        ]
 [0.31622777 0.         0.         ... 0.         0.         0.        ]]


# One-Hot-Encoding

A [one-hot-encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) is a representation of categorical variables as binary vectors. This first requires that the categorical values be mapped to interger values. Then, each interger value is represented as a binary vector that is all zero values except the index of the interger, which is marked with a 1.

In [10]:
#### one-hot encoding with scikit-learn ######
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from numpy import argmax

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(sentences_train)
print(integer_encoded)
# binary encoded
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invertz first 10 inputs
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0:10,:])])
print(inverted)

[31 31 26 41 31 41 41 47 12 29 23 29 38 26 19 41 41 41 35 26 23 41 38 31
 31 47 41 23 41 47 23 38 27 41 38 41 41 47 47 47 51 19 41 38 41 41  8 41
 16 29 39 51 38 29 26 10 10 31  9  7 47 28 29 31 47  9 23 31 38 47 31 43
 31 23 38 23 43 41 35 47 41 34 29 26 23 26 29 23  7  9 16 47 41 31 26 27
 26 23 47 31  7 41 47 10  6 28 23 29 41 23 38 16 19 31  5 41 26 23  7 31
 29 29 16  2 29  7 23 41 47 31 41 27 23 23  7 47 31 16 23 41 15  9 38 47
 31 27 49 32 41 23 41 27 29 15 29  0 26  9  7  1 31 15 47 23 31 16 15 31
 41 41 16 38 31 29  7  9 31 48 31 41 27 23  7 41 41 26 41 47 16 32 41 41
 28 32  9 47 41 14 41 31 26 27 31 41 26 14 41 26 31 19 23 31 41 31 37 47
 31  7 16 19 41 41 23 47 41 31 41 29 15 41 41  7 14 11 41 47 27 41 23 41
 24 41 41 41 41 31 16 29 23 19 23 26 19 31 41 14 31 32 15 26 47 41 38  7
  2 15 16 27  7  7 31 26 31 41 41 29 41 26 15 47 26 38 41 31 41 23 47 23
 41 29 23 38 46 41 31 16 29 41 47 26 47 23 27 29 31 31 41 41  3 19 29 16
 31 47 31 16 27 31 47 26  7 23 26 26 41 31  8  7 23

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


# Keras Tokenizer API

Keras provides a more sophisticated API for preparing text that can be fit and reused to prepare multiple text documents. This may be the preferred approach for large projects.

Keras provides the [Tokenizer class](https://keras.io/preprocessing/text/#tokenizer) for preparing text documents for deep learning. The Tokenizer must be constructed and then fit on either raw text documents or integer encoded text documents.

In [12]:
##### Tokenize using Tokenizer API #######
from keras.preprocessing.text import Tokenizer

# create the tokenizer
t = Tokenizer(
    char_level=False,
    filters="!\"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n", # will prevent ' _ ' of being removed
    lower=False
)
# fit the tokenizer on the docs
t.fit_on_texts(sentences_train)

# Vocab of words and their counts
print("Count of words: ",t.word_counts)
# Vocab of words and how many documents each appeared in
print("How many times each words appears in docs: ",t.document_count)
#Vocab of words and their uniquely assigned intergers
print("Assigned intergers: ",t.word_index)
# An interger count of the total number of documents that were used to fit the Tokenizer
print("Total nr. of docs use to tokenize: ",t.word_docs)

Count of words:  OrderedDict([('ARC_NA', 752), ('MV_CASE', 752), ('MIME_GOOD', 752), ('TO_DN_NONE', 744), ('DMARC_DNSFAIL', 752), ('FROM_NO_DN', 752), ('RCPT_COUNT_TWELVE', 131), ('RCVD_COUNT_ZERO', 752), ('MIME_TRACE', 752), ('R_DKIM_NA', 752), ('DATE_IN_PAST', 752), ('MID_RHS_NOT_FQDN', 752), ('HFILTER_HOSTNAME_UNKNOWN', 752), ('RCPT_COUNT_ONE', 261), ('TO_DOM_EQ_FROM_DOM', 209), ('RCPT_COUNT_THREE', 87), ('FAKE_REPLY', 131), ('SUBJ_ALL_CAPS', 17), ('RCPT_COUNT_SEVEN', 99), ('RCPT_COUNT_FIVE', 82), ('RCPT_COUNT_TWO', 52), ('RCPT_COUNT_GT_50', 34), ('TO_DN_SOME', 2), ('SUSPICIOUS_RECIPS', 28), ('SUBJECT_ENDS_EXCLAIM', 11), ('MISSING_TO', 13), ('INTRODUCTION', 2), ('EMPTY_SUBJECT', 7), ('SUBJECT_ENDS_QUESTION', 6), ('FREEMAIL_FROM', 2), ('FREEMAIL_ENVFROM', 2), ('SUBJ_EXCESS_QP', 1), ('SUBJECT_HAS_QUESTION', 1), ('TO_EQ_FROM', 1)])
How many times each words appears in docs:  752
Assigned intergers:  {'ARC_NA': 1, 'MV_CASE': 2, 'MIME_GOOD': 3, 'DMARC_DNSFAIL': 4, 'FROM_NO_DN': 5, 'RCVD_